 This notebook simulates a buck converter and analyzes the current and voltage waveforms, including their harmonic content through FFT analysis.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from tabulate import tabulate
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Define the BuckConverter class

class BuckConverter:
    def __init__(self, Vin=450, Vout=96, L=150e-6, C=47e-6, R=3.2, fsw=100e3):
        """Initialize the buck converter with parameters."""
        self.Vin = Vin      # Input voltage
        self.Vout = Vout    # Output voltage
        self.L = L          # Inductance
        self.C = C          # Capacitance
        self.R = R          # Load resistance
        self.fsw = fsw      # Switching frequency
        self.D = Vout / Vin # Duty cycle

    def simulate(self, t_sim_cycles=3, dt=1e-7):
        """
        Simulate the buck converter for a specified number of switching cycles.

        Args:
            t_sim_cycles: Number of switching cycles to simulate
            dt: Time step for simulation

        Returns:
            t: Time array
            currents: Dictionary of current waveforms
            voltages: Dictionary of voltage waveforms
        """
        # Time parameters
        t_sim = t_sim_cycles * (1/self.fsw)  # Simulation time
        t = np.arange(0, t_sim, dt)

        # Initialize arrays
        i_L = np.zeros_like(t)
        i_switch = np.zeros_like(t)
        i_diode = np.zeros_like(t)
        v_out = np.zeros_like(t)
        v_c = np.zeros_like(t)

        # Initial conditions - more accurate steady state
        i_L[0] = self.Vout / self.R  # Steady-state average inductor current
        v_c[0] = self.Vout           # Initial capacitor voltage
        v_out[0] = self.Vout         # Initial output voltage

        # Calculate theoretical ripple for comparison
        theoretical_ripple = self.Vin * self.D * (1 - self.D) / (self.L * self.fsw)

        # Simulation loop
        for n in range(1, len(t)):
            # Determine if switch is on or off
            if (t[n] % (1/self.fsw)) < (self.D * (1/self.fsw)):
                # Switch ON
                di_L = (self.Vin - v_out[n-1]) * dt / self.L
                i_L[n] = i_L[n-1] + di_L
                i_switch[n] = i_L[n]
                i_diode[n] = 0
            else:
                # Switch OFF
                di_L = -v_out[n-1] * dt / self.L
                i_L[n] = i_L[n-1] + di_L
                i_switch[n] = 0
                i_diode[n] = i_L[n]

            # Update capacitor voltage and output voltage
            i_C = i_L[n] - v_out[n-1] / self.R  # Capacitor current
            dv_c = i_C * dt / self.C
            v_c[n] = v_c[n-1] + dv_c
            v_out[n] = v_c[n]  # Simplified - in reality there would be ESR effects

        # Store results in dictionaries
        currents = {
            'inductor': i_L,
            'switch': i_switch,
            'diode': i_diode,
            'capacitor': i_L - v_out / self.R,
            'load': v_out / self.R
        }

        voltages = {
            'output': v_out,
            'capacitor': v_c,
            'inductor': np.zeros_like(t)  # Calculate if needed
        }

        # Calculate inductor voltage
        for n in range(len(t)):
            if i_switch[n] > 0:  # Switch ON
                voltages['inductor'][n] = self.Vin - v_out[n]
            else:  # Switch OFF
                voltages['inductor'][n] = -v_out[n]

        return t, currents, voltages, theoretical_ripple

    @staticmethod
    def perform_fft_with_peaks(signal, t, min_peak_height=0.1, num_harmonics=10):
        """
        Perform FFT with upsampling and peak detection.

        Args:
            signal: Input signal to analyze
            t: Time array
            min_peak_height: Minimum height for peak detection
            num_harmonics: Number of harmonics to detect

        Returns:
            xf: Frequency array
            magnitude_spectrum: Magnitude spectrum
            peak_freqs: Detected peak frequencies
            peak_mags: Detected peak magnitudes
        """
        # Skip transient portion (first 20% of the signal)
        skip_percent = 0.2
        start_idx = int(len(t) * skip_percent)
        t_steady = t[start_idx:]
        signal_steady = signal[start_idx:]

        # Upsampling
        t_upsampled = np.linspace(t_steady[0], t_steady[-1], len(t_steady)*10)
        interpolator = interp1d(t_steady, signal_steady, kind='cubic')
        signal_upsampled = interpolator(t_upsampled)

        N = len(signal_upsampled)
        dt_upsampled = (t_steady[-1] - t_steady[0]) / N

        # Apply window to reduce spectral leakage
        window = np.hanning(N)
        signal_windowed = signal_upsampled * window

        yf = fft(signal_windowed)
        xf = np.linspace(0.0, 1.0/(2.0*dt_upsampled), N//2)
        magnitude_spectrum = 2.0/N * np.abs(yf[0:N//2])

        # Find peaks
        peaks, _ = find_peaks(magnitude_spectrum, height=min_peak_height)
        peak_freqs = xf[peaks]
        peak_magnitudes = magnitude_spectrum[peaks]

        # Sort peaks by magnitude
        sort_idx = np.argsort(peak_magnitudes)[::-1]  # Sort in descending order
        peak_freqs = peak_freqs[sort_idx]
        peak_magnitudes = peak_magnitudes[sort_idx]

        return xf, magnitude_spectrum, peak_freqs, peak_magnitudes

    def analyze_and_plot(self, t, currents, voltages, theoretical_ripple, plot_voltages=False):
        """
        Analyze and plot the simulation results.

        Args:
            t: Time array
            currents: Dictionary of current waveforms
            voltages: Dictionary of voltage waveforms
            theoretical_ripple: Calculated theoretical ripple
            plot_voltages: Whether to also plot voltage waveforms
        """
        plt.rcParams.update({'font.size': 12})  # Sets base font size

        # Determine what to plot
        if plot_voltages:
            signals_to_plot = {
                'currents': ['inductor', 'switch', 'diode'],
                'voltages': ['output', 'capacitor', 'inductor']
            }
            num_rows = 6  # 3 currents + 3 voltages
        else:
            signals_to_plot = {
                'currents': ['inductor', 'switch', 'diode'],
                'voltages': []
            }
            num_rows = 3  # Just 3 currents

        # Create the subplots
        fig, axs = plt.subplots(num_rows, 2, figsize=(15, 5*num_rows))

        row = 0

        # Process and plot currents
        for signal_name in signals_to_plot['currents']:
            signal = currents[signal_name]
            title = f"{signal_name.capitalize()} Current"

            # Time domain plot
            axs[row, 0].plot(t*1e6, signal)
            axs[row, 0].set_xlabel('Time (µs)')
            axs[row, 0].set_ylabel('Current (A)')
            axs[row, 0].set_title(f'{title} (Time Domain)')
            axs[row, 0].grid(True)

            # Frequency domain plot with peak detection
            xf, yf, peak_freqs, peak_mags = self.perform_fft_with_peaks(signal, t)
            axs[row, 1].semilogy(xf/1e3, yf)
            axs[row, 1].set_xlabel('Frequency (kHz)')
            axs[row, 1].set_ylabel('Magnitude')
            axs[row, 1].set_title(f'{title} Spectrum')
            axs[row, 1].set_xlim(0, 500)  # Limit x-axis to 500 kHz
            axs[row, 1].grid(True)

            # Mark peaks on the plot
            axs[row, 1].plot(peak_freqs/1e3, peak_mags, 'ro')

            # Print peak analysis
            print(f"\n{title} Frequency Analysis:")

            # Calculate relative percentages
            total_magnitude = np.sum(peak_mags)
            relative_percentages = (peak_mags / total_magnitude) * 100

            # Create table data
            table_data = []
            for j in range(min(len(peak_freqs), 10)):  # Show top 10 peaks
                table_data.append([
                    f"{peak_freqs[j]/1e3:.1f}",  # Frequency in kHz
                    f"{peak_mags[j]:.3f}",       # Absolute magnitude
                    f"{relative_percentages[j]:.1f}"  # Relative percentage
                ])

            # Print table using tabulate
            headers = ["Frequency (kHz)", "Magnitude", "Relative %"]
            print(tabulate(table_data, headers=headers, tablefmt="grid"))

            row += 1

        # Process and plot voltages if requested
        for signal_name in signals_to_plot['voltages']:
            signal = voltages[signal_name]
            title = f"{signal_name.capitalize()} Voltage"

            # Time domain plot
            axs[row, 0].plot(t*1e6, signal)
            axs[row, 0].set_xlabel('Time (µs)')
            axs[row, 0].set_ylabel('Voltage (V)')
            axs[row, 0].set_title(f'{title} (Time Domain)')
            axs[row, 0].grid(True)

            # Frequency domain plot with peak detection
            xf, yf, peak_freqs, peak_mags = self.perform_fft_with_peaks(signal, t)
            axs[row, 1].semilogy(xf/1e3, yf)
            axs[row, 1].set_xlabel('Frequency (kHz)')
            axs[row, 1].set_ylabel('Magnitude')
            axs[row, 1].set_title(f'{title} Spectrum')
            axs[row, 1].set_xlim(0, 500)  # Limit x-axis to 500 kHz
            axs[row, 1].grid(True)

            # Mark peaks on the plot
            axs[row, 1].plot(peak_freqs/1e3, peak_mags, 'ro')

            row += 1

        plt.tight_layout()
        plt.show()

        # Calculate and print ripple currents and voltages
        i_L = currents['inductor']
        v_out = voltages['output']

        # Skip first 20% to avoid transient
        start_idx = int(len(t) * 0.2)
        i_L_steady = i_L[start_idx:]
        v_out_steady = v_out[start_idx:]

        i_L_max = np.max(i_L_steady)
        i_L_min = np.min(i_L_steady)
        i_L_ripple = i_L_max - i_L_min

        v_out_max = np.max(v_out_steady)
        v_out_min = np.min(v_out_steady)
        v_out_ripple = v_out_max - v_out_min

        print("\n--- Buck Converter Performance Summary ---")
        print(f"Inductor current ripple (simulated): {i_L_ripple:.3f} A")
        print(f"Inductor current ripple (theoretical): {theoretical_ripple:.3f} A")
        print(f"Output voltage ripple: {v_out_ripple:.3f} V ({v_out_ripple/self.Vout*100:.2f}%)")

        # Calculate and print average currents and voltages
        i_L_avg = np.mean(i_L_steady)
        i_switch_avg = np.mean(currents['switch'][start_idx:])
        i_diode_avg = np.mean(currents['diode'][start_idx:])
        v_out_avg = np.mean(v_out_steady)

        print("\n--- Average Values ---")
        print(f"Average inductor current: {i_L_avg:.3f} A")
        print(f"Average switch current: {i_switch_avg:.3f} A")
        print(f"Average diode current: {i_diode_avg:.3f} A")
        print(f"Average output voltage: {v_out_avg:.3f} V")

        # Power calculations
        p_in_avg = i_switch_avg * self.Vin
        p_out_avg = v_out_avg**2 / self.R
        efficiency = p_out_avg / p_in_avg * 100 if p_in_avg > 0 else 0

        print("\n--- Power Analysis ---")
        print(f"Input power: {p_in_avg:.3f} W")
        print(f"Output power: {p_out_avg:.3f} W")
        print(f"Efficiency: {efficiency:.2f}%")

Basic usage example



In [ ]:
# Basic usage example with JSON export
# ====================================
# Run this cell to simulate, analyze the buck converter, and export the results to JSON

import json

# Create a buck converter with default parameters
buck = BuckConverter()

# Simulate
t, currents, voltages, theoretical_ripple = buck.simulate()

# Dictionary to store the peaks and amplitudes
harmonic_data = {
    'parameters': {
        'Vin': buck.Vin,
        'Vout': buck.Vout,
        'L': buck.L,
        'C': buck.C,
        'R': buck.R,
        'fsw': buck.fsw,
        'duty_cycle': buck.D
    },
    'harmonics': {}
}

# Process current waveforms and store peaks
for signal_name in ['inductor', 'switch', 'diode']:
    signal = currents[signal_name]
    xf, yf, peak_freqs, peak_mags = buck.perform_fft_with_peaks(signal, t)

    # Calculate relative percentages
    total_magnitude = np.sum(peak_mags)
    relative_percentages = (peak_mags / total_magnitude) * 100

    # Store the frequency peaks data
    harmonics = []
    for j in range(min(len(peak_freqs), 10)):  # Store top 10 peaks
        harmonics.append({
            'frequency_khz': float(peak_freqs[j])/1e3,
            'magnitude': float(peak_mags[j]),
            'relative_percentage': float(relative_percentages[j])
        })

    harmonic_data['harmonics'][f'{signal_name}_current'] = harmonics

# Process voltage waveforms if needed
for signal_name in ['output', 'capacitor', 'inductor']:
    signal = voltages[signal_name]
    xf, yf, peak_freqs, peak_mags = buck.perform_fft_with_peaks(signal, t)

    # Calculate relative percentages
    total_magnitude = np.sum(peak_mags)
    relative_percentages = (peak_mags / total_magnitude) * 100

    # Store the frequency peaks data
    harmonics = []
    for j in range(min(len(peak_freqs), 10)):  # Store top 10 peaks
        harmonics.append({
            'frequency_khz': float(peak_freqs[j])/1e3,
            'magnitude': float(peak_mags[j]),
            'relative_percentage': float(relative_percentages[j])
        })

    harmonic_data['harmonics'][f'{signal_name}_voltage'] = harmonics

# Add performance metrics
# Skip first 20% to avoid transient
start_idx = int(len(t) * 0.2)
i_L_steady = currents['inductor'][start_idx:]
v_out_steady = voltages['output'][start_idx:]

i_L_ripple = np.max(i_L_steady) - np.min(i_L_steady)
v_out_ripple = np.max(v_out_steady) - np.min(v_out_steady)

i_switch_avg = np.mean(currents['switch'][start_idx:])
v_out_avg = np.mean(v_out_steady)

p_in_avg = i_switch_avg * buck.Vin
p_out_avg = v_out_avg**2 / buck.R
efficiency = p_out_avg / p_in_avg * 100 if p_in_avg > 0 else 0

harmonic_data['performance'] = {
    'inductor_current_ripple': float(i_L_ripple),
    'theoretical_ripple': float(theoretical_ripple),
    'output_voltage_ripple': float(v_out_ripple),
    'output_voltage_ripple_percent': float(v_out_ripple/buck.Vout*100),
    'average_inductor_current': float(np.mean(i_L_steady)),
    'average_switch_current': float(i_switch_avg),
    'average_diode_current': float(np.mean(currents['diode'][start_idx:])),
    'average_output_voltage': float(v_out_avg),
    'input_power': float(p_in_avg),
    'output_power': float(p_out_avg),
    'efficiency': float(efficiency)
}

# Convert to JSON and print (can be copied to clipboard)
json_data = json.dumps(harmonic_data, indent=2)
print("Harmonic Analysis Results (JSON format):")
print(json_data)

# Optionally save to a file
with open('buck_converter_harmonics.json', 'w') as f:
    f.write(json_data)
print("\nResults also saved to 'buck_converter_harmonics.json'")

# Also perform the standard analysis and plotting
buck.analyze_and_plot(t, currents, voltages, theoretical_ripple)


**Functions below are still experimental**

In [ ]:
# Interactive parameter adjustment using ipywidgets
# Run this cell to create interactive sliders for adjusting buck converter parameters

def run_simulation(Vin, Vout, L, C, R, fsw, cycles, plot_voltages):
    # Create the buck converter with the specified parameters
    buck = BuckConverter(Vin, Vout, L*1e-6, C*1e-6, R, fsw*1e3)

    # Simulate
    t, currents, voltages, theoretical_ripple = buck.simulate(cycles)

    # Clear previous output
    clear_output(wait=True)

    # Display the current settings
    print(f"Vin = {Vin} V, Vout = {Vout} V, L = {L} µH, C = {C} µF, R = {R} Ω, fsw = {fsw} kHz")
    print(f"Duty cycle = {buck.D:.3f}, Cycles = {cycles}")

    # Analyze and plot
    buck.analyze_and_plot(t, currents, voltages, theoretical_ripple, plot_voltages)

# Create interactive widgets
Vin_slider = widgets.FloatSlider(value=450, min=100, max=600, step=10, description='Vin (V)')
Vout_slider = widgets.FloatSlider(value=96, min=5, max=500, step=5, description='Vout (V)')
L_slider = widgets.FloatSlider(value=150, min=10, max=500, step=10, description='L (µH)')
C_slider = widgets.FloatSlider(value=47, min=1, max=200, step=1, description='C (µF)')
R_slider = widgets.FloatSlider(value=3.2, min=0.1, max=20, step=0.1, description='R (Ω)')
fsw_slider = widgets.FloatSlider(value=100, min=10, max=500, step=10, description='fsw (kHz)')
cycles_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Cycles')
plot_voltages_checkbox = widgets.Checkbox(value=False, description='Plot Voltages')

# Create interactive output
interactive_output = widgets.interactive_output(
    run_simulation,
    {
        'Vin': Vin_slider,
        'Vout': Vout_slider,
        'L': L_slider,
        'C': C_slider,
        'R': R_slider,
        'fsw': fsw_slider,
        'cycles': cycles_slider,
        'plot_voltages': plot_voltages_checkbox
    }
)

# Display the widgets and output
display(widgets.VBox([
    widgets.HBox([Vin_slider, Vout_slider]),
    widgets.HBox([L_slider, C_slider]),
    widgets.HBox([R_slider, fsw_slider]),
    widgets.HBox([cycles_slider, plot_voltages_checkbox])
]))
display(interactive_output)

In [ ]:
# Advanced Analysis: Parameter Sweep
# This cell allows you to analyze how one parameter affects the converter performance
# while keeping all other parameters constant

def parameter_sweep(param_name, param_values, fixed_params=None):
    """
    Perform a parameter sweep and analyze the results.

    Args:
        param_name: Name of the parameter to sweep ('Vin', 'L', etc.)
        param_values: List of values to try for the parameter
        fixed_params: Dictionary of fixed parameters (defaults to standard values if None)
    """
    # Default parameters
    default_params = {
        'Vin': 450,
        'Vout': 96,
        'L': 150e-6,
        'C': 47e-6,
        'R': 3.2,
        'fsw': 100e3,
        'cycles': 3
    }

    # Update with fixed parameters if provided
    if fixed_params:
        default_params.update(fixed_params)

    # Results storage
    results = {
        'param_values': param_values,
        'ripple_current': [],
        'ripple_voltage': [],
        'efficiency': []
    }

    # Run simulations for each parameter value
    for value in param_values:
        # Create a copy of the parameters
        params = default_params.copy()

        # Update the parameter to sweep
        params[param_name] = value

        # Create and simulate the buck converter
        buck = BuckConverter(
            params['Vin'],
            params['Vout'],
            params['L'],
            params['C'],
            params['R'],
            params['fsw']
        )

        t, currents, voltages, theoretical_ripple = buck.simulate(params['cycles'])

        # Calculate performance metrics
        i_L = currents['inductor']
        v_out = voltages['output']

        # Skip first 20% to avoid transient
        start_idx = int(len(t) * 0.2)
        i_L_steady = i_L[start_idx:]
        v_out_steady = v_out[start_idx:]

        i_L_ripple = np.max(i_L_steady) - np.min(i_L_steady)
        v_out_ripple = np.max(v_out_steady) - np.min(v_out_steady)

        i_switch_avg = np.mean(currents['switch'][start_idx:])
        v_out_avg = np.mean(v_out_steady)

        p_in_avg = i_switch_avg * params['Vin']
        p_out_avg = v_out_avg**2 / params['R']
        efficiency = p_out_avg / p_in_avg * 100 if p_in_avg > 0 else 0

        # Store results
        results['ripple_current'].append(i_L_ripple)
        results['ripple_voltage'].append(v_out_ripple)
        results['efficiency'].append(efficiency)

    # Plot results
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Ripple current on left y-axis
    color = 'tab:blue'
    ax1.set_xlabel(f'{param_name} Value')
    ax1.set_ylabel('Ripple Current (A)', color=color)
    ax1.plot(results['param_values'], results['ripple_current'], 'o-', color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    # Ripple voltage on right y-axis
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Ripple Voltage (V)', color=color)
    ax2.plot(results['param_values'], results['ripple_voltage'], 's-', color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    # Efficiency on a second plot
    fig.tight_layout()
    plt.grid(True)
    plt.title(f'Effect of {param_name} on Ripple Current and Voltage')

    plt.figure(figsize=(10, 4))
    plt.plot(results['param_values'], results['efficiency'], 'o-', color='tab:green')
    plt.xlabel(f'{param_name} Value')
    plt.ylabel('Efficiency (%)')
    plt.grid(True)
    plt.title(f'Effect of {param_name} on Efficiency')

    plt.tight_layout()
    plt.show()

    return results

# Example: Sweep inductor values
inductor_values = [50e-6, 100e-6, 150e-6, 200e-6, 250e-6, 300e-6]
# Uncomment to run the sweep
# results = parameter_sweep('L', inductor_values)

# Example: Sweep switching frequency
# frequencies = [50e3, 100e3, 150e3, 200e3, 250e3, 300e3]
# results = parameter_sweep('fsw', frequencies)